In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, normalize

In [2]:
from xgboost import XGBRegressor, XGBRFRegressor

In [3]:
features = pd.read_csv("features.csv")
stores = pd.read_csv("stores.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [4]:
features.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [5]:
features.shape

(8190, 12)

In [6]:
features.head(3)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False


In [7]:
stores.head(3)

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392


In [8]:
train.head(3)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False


In [9]:
raw_df = train.merge(stores, on="Store")
raw_df = raw_df.merge(features, on=["Date", "Store"])

In [10]:
raw_df.isna().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Type                 0
Size                 0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday_y          0
dtype: int64

In [11]:
raw_df.drop(["MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5", "IsHoliday_y"], axis=1, inplace=True)

In [12]:
raw_df.isna().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday_x     0
Type            0
Size            0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [13]:
# Form features and target
features = raw_df.columns.tolist()
target = features.pop(features.index("Weekly_Sales"))

In [14]:
raw_df[features].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   IsHoliday_x   421570 non-null  bool   
 4   Type          421570 non-null  object 
 5   Size          421570 non-null  int64  
 6   Temperature   421570 non-null  float64
 7   Fuel_Price    421570 non-null  float64
 8   CPI           421570 non-null  float64
 9   Unemployment  421570 non-null  float64
dtypes: bool(1), float64(4), int64(3), object(2)
memory usage: 32.6+ MB


In [15]:
raw_df["Date"] = pd.to_datetime(raw_df["Date"])

In [16]:
raw_df.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,211.096358,8.106
1,1,2,2010-02-05,50605.27,False,A,151315,42.31,2.572,211.096358,8.106


In [17]:
# Remove duplicates
raw_df.drop_duplicates(inplace=True)

In [18]:
raw_df.IsHoliday_x = raw_df.IsHoliday_x.map(lambda x: 1 if x == True else 0)

In [19]:
cat_cols = ["Store", "Dept", "IsHoliday_x", "Type"]
num_cols = ["Size", "Temperature", "Fuel_Price", "CPI", "Unemployment"]

In [20]:
for col in cat_cols:
    raw_df[col] = raw_df[col].astype('category')

In [21]:
pd.get_dummies(raw_df[cat_cols])

,Store_1,Store_2,Store_3,Store_4,Store_5,Store_6,Store_7,Store_8,Store_9,Store_10,...,Dept_95,Dept_96,Dept_97,Dept_98,Dept_99,IsHoliday_x_0,IsHoliday_x_1,Type_A,Type_B,Type_C
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
421566,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
421567,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
421568,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
